In [10]:
from tabula import read_pdf
import pandas as pd

# Especificar la ruta del archivo PDF
file_path = "N~100 I.E NORMAL SUPERIOR NUESTRA SEÑORA DE FATIMA.pdf"

# Extraer tablas como DataFrame
dataframes = read_pdf(file_path, pages="all", multiple_tables=True)

# Manipular tablas (Ejemplo para la primera tabla extraída)
if dataframes:
    df = dataframes[0]
    print(df.head())
else:
    print("No se encontraron tablas en el PDF.")


KeyboardInterrupt: 

In [11]:
from pdf2image import convert_from_path
import pytesseract
import pandas as pd
import numpy as np
import cv2
import os

# Rutas / configuracion
pdf_path = "N~113 LOS CLAVELES.pdf"
poppler_path = "/usr/bin"
pytesseract.pytesseract.tesseract_cmd = "/usr/bin/tesseract"
os.environ["TESSDATA_PREFIX"] = "/usr/share/tessdata/spa.traineddata"

# Convertir los pdf en imagenes
imagenes = convert_from_path(pdf_path, poppler_path = poppler_path)

# Proceso por paginas
dfs = []
for i, img in enumerate(imagenes):
    # Convertir a lista de numpy
    img_array = np.array(img)
    # Convertir a escala de grises y Binarizar imagen
    gray = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV)

    #Encontrar contornos para demarcar las tablas
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Se demarcan las regiones de la tabla
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        table_img = img_array[y:y + h, x:x + w]

        #ocr sobre la region de la tabla para mostrar el texto
        table_text = pytesseract.image_to_string(table_img, config = "--psm 6")
        
        #Convertir texto tabulado a DataFrame
        rows = table_text.split("\n")
        table_data = [row.split() for row in rows if row.strip()]
        df = pd.DataFrame(table_data)

        #se agrega el data frame a la lista ya creada
        dfs.append(df)

# Se combinan los dataframes
final_df = pd.cocat(dfs, ignore_index = True)
print(final_df)


TesseractError: (1, 'Error opening data file /usr/share/tessdata/spa.traineddata/eng.traineddata Please make sure the TESSDATA_PREFIX environment variable is set to your "tessdata" directory. Failed loading language \'eng\' Tesseract couldn\'t load any languages! Could not initialize tesseract.')

In [12]:
from pdf2image import convert_from_path
import pytesseract
import pandas as pd
import numpy as np
import cv2
from pathlib import Path

# Configuración de rutas
pdf_path = Path("N~113 LOS CLAVELES.pdf")
poppler_path = "/usr/bin"
pytesseract.pytesseract.tesseract_cmd = "/usr/bin/tesseract"
tessdata_prefix = "/usr/share/tessdata"

# Validar existencia del archivo PDF
if not pdf_path.exists():
    raise FileNotFoundError(f"El archivo PDF '{pdf_path}' no existe.")

# Convertir el PDF en imágenes
imagenes = convert_from_path(str(pdf_path), poppler_path=poppler_path)

# Procesamiento por página
dfs = []
for i, img in enumerate(imagenes):
    # Convertir la imagen a array numpy
    img_array = np.array(img)
    # Convertir a escala de grises y aplicar binarización adaptativa
    gray = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
    binary = cv2.adaptiveThreshold(
        gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2
    )

    # Encontrar contornos para detectar tablas
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Procesar cada contorno detectado
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        # Filtro para evitar cuadros muy pequeños (ruido)
        if w < 50 or h < 50:
            continue
        
        table_img = img_array[y:y + h, x:x + w]

        # Aplicar OCR sobre la región seleccionada
        custom_config = "--psm 6 -c tessedit_char_whitelist='0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ,.'"
        table_text = pytesseract.image_to_string(table_img, config=custom_config)

        # Procesar texto y convertirlo a un DataFrame
        rows = table_text.strip().split("\n")
        table_data = [row.split() for row in rows if row.strip()]
        
        if table_data:  # Asegurarse de no agregar tablas vacías
            df = pd.DataFrame(table_data)
            dfs.append(df)

# Combinar los DataFrames si hay datos
if dfs:
    final_df = pd.concat(dfs, ignore_index=True)
    print(final_df)
else:
    print("No se encontraron tablas en el PDF.")


TesseractError: (1, 'Error opening data file /usr/share/tessdata/spa.traineddata/eng.traineddata Please make sure the TESSDATA_PREFIX environment variable is set to your "tessdata" directory. Failed loading language \'eng\' Tesseract couldn\'t load any languages! Could not initialize tesseract.')

In [ ]:
from pdf2image import convert_from_path
import pytesseract
import pandas as pd
import numpy as np
import cv2
from pathlib import Path
import os

# Configuración de rutas
pdf_path = Path("N~100 I.E NORMAL SUPERIOR NUESTRA SEÑORA DE FATIMA.pdf")
poppler_path = "/usr/bin"
pytesseract.pytesseract.tesseract_cmd = "/usr/bin/tesseract"
os.environ["TESSDATA_PREFIX"] = "/usr/share/tessdata"  # Ajusta esta ruta si es diferente

# Validar existencia del archivo PDF
if not pdf_path.exists():
    raise FileNotFoundError(f"El archivo PDF '{pdf_path}' no existe.")

# Convertir el PDF en imágenes
imagenes = convert_from_path(str(pdf_path), poppler_path=poppler_path)

# Procesamiento por página
dfs = []
for i, img in enumerate(imagenes):
    # Convertir la imagen a array numpy
    img_array = np.array(img)
    # Convertir a escala de grises y aplicar binarización adaptativa
    gray = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
    binary = cv2.adaptiveThreshold(
        gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2
    )

    # Encontrar contornos para detectar tablas
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Procesar cada contorno detectado
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        # Filtro para evitar cuadros muy pequeños (ruido)
        if w < 50 or h < 50:
            continue
        
        table_img = img_array[y:y + h, x:x + w]

        # Aplicar OCR sobre la región seleccionada
        custom_config = "--psm 6 -l spa"  # Cambia a 'eng' para inglés
        table_text = pytesseract.image_to_string(table_img, config=custom_config)

        # Procesar texto y convertirlo a un DataFrame
        rows = table_text.strip().split("\n")
        table_data = [row.split() for row in rows if row.strip()]
        
        if table_data:  # Asegurarse de no agregar tablas vacías
            df = pd.DataFrame(table_data)
            dfs.append(df)

# Combinar los DataFrames si hay datos
if dfs:
    final_df = pd.concat(dfs, ignore_index=True)
    final_df
else:
    print("No se encontraron tablas en el PDF.")


In [15]:
final_df

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,E,A,None,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,y,E,m-u,El,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1E,EE,Es,E,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,E,E,E,rel,ei,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,o,|m,o],20,E,Es,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,—|,foemos,Jueor,—,fpmaue,f,fuez,|7,|,|,...,P,Y,Y,T,Esl,None,None,None,None,None
111,e,jeees,jue,—,paeas,queeno,qeomoo,|<|,|,|o|amjafaix]a,...,aI,T,TEEITILILIIS,None,None,None,None,None,None,None
112,|,femas,quens,qoe,—,qroosaie,fuee,|,|,|,...,p,>,aa,P,P,Y,EE,Y,asl,None
113,e,[e,fess,fueo,-,fea,—,j,famevo|>,|,...,|y,e,a<,p,AE,EE,sl,None,None,None
